In [1]:
import os
import sys
import numpy as np
import cv2

from data_loader import *
from fbs_config import TrainFBSConfig, InferenceFBSConfig
from fbs_dataset import FBSDataset

from mrcnn import model as modellib
from datahandler import DataHandler

from sklearn.metrics import f1_score
from scipy.ndimage import _ni_support
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion,\
     generate_binary_structure
from tqdm import tqdm

import skimage.color

ROOT_DIR = os.path.abspath('../../../')
sys.path.append(ROOT_DIR)

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, 'logs')
DEFAULT_MODEL_DIR = os.path.join(DEFAULT_LOGS_DIR, 'mask_rcnn/kfold')

kernel = np.ones((5,5),np.uint8)

dh = DataHandler()

Using TensorFlow backend.


In [2]:
image_files, mask_files = load_data_files('data/kfold_data/')

skf = getKFolds(image_files, mask_files, n=10)

kfold_indices = []
for train_index, test_index in skf.split(image_files, mask_files):
    kfold_indices.append({'train': train_index, 'val': test_index})

In [3]:
def getDataset(val_index):
    image_val_files = np.take(image_files, val_index)
    mask_val_files = np.take(mask_files, val_index)

    val_files = ([image_val_files], [mask_val_files])

    dataset_val = FBSDataset()
    len_dataset_val = dataset_val.load_data(val_files)
    dataset_val.prepare()
    return dataset_val

In [4]:
def getDiceScore(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)

In [5]:
 def hd(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

def hd95(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity)
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity)
    hd95 = np.percentile(np.hstack((hd1, hd2)), 95)
    return hd95

def __surface_distances(result, reference, voxelspacing=None, connectivity=1):
    result = np.atleast_1d(result.astype(np.bool))
    reference = np.atleast_1d(reference.astype(np.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = np.asarray(voxelspacing, dtype=np.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()

    footprint = generate_binary_structure(result.ndim, connectivity)

    if 0 == np.count_nonzero(result):
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == np.count_nonzero(reference):
        raise RuntimeError('The second supplied array does not contain any binary object.')

    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)

    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]

    return sds

In [6]:
def evaluateMask(gt_mask, pred_mask):
    return getDiceScore(gt_mask, pred_mask), hd(gt_mask, pred_mask), hd95(gt_mask, pred_mask)
    

In [7]:
import random
def predictAll(inferenceFBSConfig, val_indices):
    model = modellib.MaskRCNN(mode='inference', config=inferenceFBSConfig, model_dir=DEFAULT_MODEL_DIR)
    inferenceFBSConfig.display()
    print(DEFAULT_MODEL_DIR)
    weights_path = model.find_last()
    print('Loading weights from %s'%weights_path)
    model.load_weights(weights_path, by_name=True)
    
    dice_scores = []
    hd_scores = []
    hd95_scores = []
    
    for image_index in tqdm(val_indices):        
        dataset = getDataset(image_index)
                
        prediction = []
        gt_mask = []
    
        for img_id in dataset.image_ids:
            image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
            dataset, inferenceFBSConfig, img_id, use_mini_mask=False)
            
            results = model.detect([image], verbose=0)
            r = results[0]
            
            pred = r['masks']
            
            if(len(pred.shape) > 2 and pred.shape[2] == 0):
                pred = np.zeros((pred.shape[0],pred.shape[1],1))
            
            if(mask.shape[2] == 0):
                mask = np.zeros((pred.shape[0],pred.shape[1],1))
            
            pred = np.asarray(pred, dtype=np.uint8)
            pred = cv2.dilate(pred,kernel,iterations = 1)
                     
            prediction.append(pred)
            gt_mask.append(mask)
        
        pred_mask = np.asarray(prediction, dtype=np.bool)
        gt_mask = np.asarray(gt_mask, dtype=np.bool)
        
        dice_score, hd_score, hd95_score = evaluateMask(np.squeeze(gt_mask), pred_mask)
    
        dice_scores.append(dice_score)
        hd_scores.append(hd_score)
        hd95_scores.append(hd95_score)
    
    return dice_scores, hd_scores, hd95_scores

In [8]:
all_dice = []
all_hd = []
all_hd95 = []

for i in range(1):#len(kfold_indices)):    
    
    configParams = {'da': False,'tl': False, 'mask_dim': 56, 'wl': True, 'kfold_i': i}

    trainFBSConfig = TrainFBSConfig(**configParams)
    inferenceFBSConfig = InferenceFBSConfig(**configParams)    
    
    dice_scores, hd_scores, hd95_scores = predictAll(inferenceFBSConfig, kfold_indices[i]['val'])
    
    print('K%d results'%i)
    print('dice %f'%np.mean(dice_scores))
    print('hd %f'%np.mean(hd_scores))
    print('hd95 %f'%np.mean(hd95_scores))
    print()
    
    '''all_dice.append(dice_score)
    all_hd.append(hd_score)
    all_hd95.append(hd95_score)'''


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   1]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 64
MASK_SHAPE                  

loading:   0%|          | 0/1 [00:00<?, ?it/s]

Re-starting from epoch 10



100%|██████████| 30/30 [02:29<00:00,  4.49s/it]08it/s]

K0 results
dice 0.822218
hd 61.597217
hd95 33.244203



In [9]:
print('dice')
for score in dice_scores:
    print(score)

print()
print('hd')
for score in hd_scores:
    print(score)

print()
print('hd95')
for score in hd95_scores:
    print(score)
    

dice
0.838611370809035
0.848912122520103
0.7898076939863997
0.7634641710033611
0.6500605784340776
0.906038089610167
0.9119834098652052
0.8351184680298605
0.8960641741333208
0.8804653725313069
0.9052908881091981
0.8905572573300973
0.7912089409840535
0.9167329716696806
0.9296369224154359
0.8275005925812842
0.8260704812428951
0.6880523910723568
0.642115651457701
0.8878144633761667
0.10235124721478157
0.911867138070561
0.9071939246741696
0.8989229141393196
0.8946129128777531
0.8600156622318261
0.8843393500983838
0.909027606864799
0.8225796801682066
0.8501319135220312

hd
17.88854381999832
98.33615815151617
110.17259187293362
23.62202362203543
122.08603523745047
70.69653456853455
17.233687939614086
36.08323710533743
28.705400188814647
42.0
93.96275858019496
107.09341716464182
51.61395160225576
57.0
47.80167361086848
23.790754506740637
74.73954776421918
116.02154972245458
134.929611279363
65.86349520030045
132.07952150125317
24.289915602982237
143.11184437355283
49.98999899979995
67.08203932